Imports

In [22]:
import csv
import os
from together import Together
from dotenv import load_dotenv
import pandas as pd

Initial Prompt

In [23]:
prompt="""
You are a specialist in human-AI detection and have extensive experience in distinguishing between texts written by humans and those generated by AI.         
You will receive a CSV-formatted dataset with two columns: ID and Text. Each row is an entry containing an ID and a text passage. Your task is to classify each entry into one of two classes:
- Human: if you believe the text is written by a human.
- AI: if you believe the text is generated by an AI.

Your response must strictly follow this CSV format:
ID;Label
Use the same ID from the input. Only output the CSV rows without explanations, comments, or extra formatting. The separator is a semicolon `;`.

### Example Input:
ID;Text  
D3-1;[Text]  
D3-2;[Text]
### Example Output:
D3-1;Human  
D3-2;AI
---
"""

Dataset to Predict

In [24]:
fileContent = ""
with open("../../datasets/submission3_inputs.csv", mode='r', encoding='utf-8') as file:
    fileContent = file.read()

prompt += "\n### Here is the input dataset:\n" + fileContent 

Zero Shot

In [25]:
load_dotenv()
client = Together(
    api_key = os.getenv("DEEPSEEK_KEY")
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3",
    messages=[{"role": "user", "content": prompt}],
)

results = response.choices[0].message.content.replace(" ", "").split('\n')

Create Predictions File

In [26]:
parsed_results = [row.split(';') for row in results if ';' in row]

# Ensure ID and Label are correctly extracted
ids = [row[0] for row in parsed_results]
labels = [row[1] for row in parsed_results]

# Create output DataFrame with ID and predictions
output_new_df = pd.DataFrame({
    "ID": ids,
    "Label": labels
})
# Save the predictions to a new CSV file
output_new_df.to_csv("results-s2.csv", sep="\t", index=False)